# 🧹 Data Cleaning — Amazon Sales Dataset

This notebook identifies data quality issues, exports bad rows to a separate CSV, cleans the main dataset, and re-runs the validation pipeline.

> ⚠️ **Run this notebook BEFORE running `python dq_pipeline.py`** to ensure the data is clean.

## 1. Load Data

In [1]:
import pandas as pd
import importlib

CSV_PATH = "data/amazon_sales.csv"
BAD_ROWS_PATH = "data/bad_rows.csv"

df = pd.read_csv(CSV_PATH, low_memory=False)
print(f"Rows: {len(df):,}  |  Columns: {len(df.columns)}")
df.head()

Rows: 128,975  |  Columns: 24


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


## 2. Inspect Data Quality Issues

In [2]:
key_cols = ["Order ID", "Date", "Status", "Fulfilment", "currency", "Qty", "Amount", "ship-country"]

print("=== NULL COUNTS ===")
null_counts = df[key_cols].isnull().sum()
print(null_counts[null_counts > 0])
print()
print("=== All Status Values ===")
print(df["Status"].value_counts(dropna=False))
print()
print("=== Currency Values ===")
print(df["currency"].value_counts(dropna=False))
print()
print("=== Ship-Country Values ===")
print(df["ship-country"].value_counts(dropna=False))

=== NULL COUNTS ===
currency        7795
Amount          7795
ship-country      33
dtype: int64

=== All Status Values ===
Status
Shipped                          77804
Shipped - Delivered to Buyer     28769
Cancelled                        18332
Shipped - Returned to Seller      1953
Shipped - Picked Up                973
Pending                            658
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipping                             8
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64

=== Currency Values ===
currency
INR    121180
NaN      7795
Name: count, dtype: int64

=== Ship-Country Values ===
ship-country
IN     128942
NaN        33
Name: count, dtype: int64


## 3. Identify & Export Bad Rows

In [3]:
# Identify all rows with any issue
mask_null_currency = df["currency"].isnull()
mask_null_amount = df["Amount"].isnull()
mask_null_country = df["ship-country"].isnull()
mask_null_order_id = df["Order ID"].isnull()
mask_neg_qty = df["Qty"] < 0

bad_mask = mask_null_currency | mask_null_amount | mask_null_country | mask_null_order_id | mask_neg_qty

bad_rows = df[bad_mask].copy()
bad_rows["issue"] = ""
bad_rows.loc[mask_null_currency, "issue"] += "null_currency; "
bad_rows.loc[mask_null_amount, "issue"] += "null_amount; "
bad_rows.loc[mask_null_country, "issue"] += "null_ship_country; "
bad_rows.loc[mask_null_order_id, "issue"] += "null_order_id; "
bad_rows.loc[mask_neg_qty, "issue"] += "negative_qty; "

print(f"Total bad rows found: {len(bad_rows):,}")
print(f"  Null currency:     {mask_null_currency.sum():,}")
print(f"  Null Amount:       {mask_null_amount.sum():,}")
print(f"  Null ship-country: {mask_null_country.sum():,}")
print(f"  Null Order ID:     {mask_null_order_id.sum():,}")
print(f"  Negative Qty:      {mask_neg_qty.sum():,}")
print()
bad_rows.head(10)

Total bad rows found: 7,826
  Null currency:     7,795
  Null Amount:       7,795
  Null ship-country: 33
  Null Order ID:     0
  Negative Qty:      0



,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22,issue
8,8,407-5443024-5233168,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,...,NaN,HYDERABAD,TELANGANA,500008.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN,null_currency; null_amount;
29,29,404-5933402-8801952,04-30-22,Cancelled,Merchant,Amazon.in,Standard,JNE2132,JNE2132-KR-398-XXXL,kurta,...,NaN,GUWAHATI,ASSAM,781003.0,IN,NaN,False,Easy Ship,NaN,null_currency; null_amount;
65,65,171-4137548-0481151,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,JNE3373,JNE3373-KR-XXL,kurta,...,NaN,Dahod,Gujarat,389151.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
84,84,403-9950518-0349133,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,JNE3510,JNE3510-KR-M,kurta,...,NaN,HYDERABAD,TELANGANA,500072.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
95,95,405-9112089-3379536,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,JNE3405,JNE3405-KR-L,kurta,...,NaN,PUNE,MAHARASHTRA,411046.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
101,101,402-7841951-1872363,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,SET347,SET347-KR-NP-S,Set,...,NaN,NEW DELHI,DELHI,110036.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
132,132,171-0293372-4913973,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,SET269,SET269-KR-NP-XXXL,Set,...,NaN,KOLAR,KARNATAKA,563101.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
139,139,403-6442534-2769112,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,J0300,J0300-TP-XXXL,Top,...,NaN,AZAMGARH,UTTAR PRADESH,276302.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
155,155,402-8078642-8565124,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,JNE3887,JNE3887-KR-XL,kurta,...,NaN,Puri-2,ODISHA,752002.0,IN,NaN,False,NaN,NaN,null_currency; null_amount;
165,165,406-0218328-1773963,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,PJNE2100,PJNE2100-KR-N-6XL,kurta,...,NaN,WALAJAPET,TAMIL NADU,632513.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN,null_currency; null_amount;


In [4]:
# Export bad rows to a separate CSV for reference
bad_rows.to_csv(BAD_ROWS_PATH, index=False)
print(f"✅ Exported {len(bad_rows):,} bad rows → {BAD_ROWS_PATH}")

✅ Exported 7,826 bad rows → data/bad_rows.csv


## 4. Fix Data Issues

| Fix | Column | Action | Reason |
|-----|--------|--------|--------|
| 1 | `currency` | Fill NaN → `"INR"` | All valid rows use INR |
| 2 | `Amount` | Fill NaN → `0.0` | Cancelled orders have no amount |
| 3 | `ship-country` | Fill NaN → `"IN"` | All valid rows use IN |

In [5]:
# Apply fixes
df["currency"]     = df["currency"].fillna("INR")
df["Amount"]       = df["Amount"].fillna(0.0)
df["ship-country"] = df["ship-country"].fillna("IN")

print("✅ All fixes applied!")
print()
print("Remaining nulls in key columns:")
remaining = df[key_cols].isnull().sum()
remaining = remaining[remaining > 0]
print("  None! ✅" if remaining.empty else remaining)

✅ All fixes applied!

Remaining nulls in key columns:
  None! ✅


## 5. Save Cleaned Data

In [6]:
df.to_csv(CSV_PATH, index=False)
print(f"✅ Cleaned data saved → {CSV_PATH}")
print(f"   {len(df):,} rows  |  {len(df.columns)} columns")

✅ Cleaned data saved → data/amazon_sales.csv
   128,975 rows  |  24 columns


## 6. Re-run Validation Pipeline

Reload the modules to pick up any code changes, then validate the cleaned data.

In [7]:
# Force reload modules (picks up code changes without kernel restart)
import src.ge_validation as _ge
import src.pydantic_validation as _py
importlib.reload(_ge)
importlib.reload(_py)

# Re-read the cleaned CSV
df_clean = pd.read_csv(CSV_PATH, low_memory=False)

print("=" * 60)
print("   RE-RUNNING VALIDATION ON CLEANED DATA")
print("=" * 60)

ge_summary = _ge.run_ge_validation(df_clean)
pydantic_summary = _py.run_pydantic_validation(df_clean)

all_ok = ge_summary["overall_success"] and pydantic_summary["overall_success"]

print("\n" + "=" * 60)
print("   FINAL RESULT")
print("=" * 60)
print(f"   GE Validation      : {'✅' if ge_summary['overall_success'] else '❌'}")
print(f"   Pydantic Validation : {'✅' if pydantic_summary['overall_success'] else '❌'}")
print(f"   Overall             : {'✅ ALL PASSED' if all_ok else '❌ ISSUES FOUND'}")
print("=" * 60)

   RE-RUNNING VALIDATION ON CLEANED DATA

🔍 Running Great Expectations Validation...


Calculating Metrics: 100%|██████████| 60/60 [00:01<00:00, 45.13it/s] 



📊 Processing GE Results...

   GE VALIDATION SUMMARY
   Overall : ✅ PASSED
   Total   : 8
   Passed  : 8
   Failed  : 0

✅ PASSED EXPECTATIONS:
------------------------------------------------------------
   ✓ expect_column_values_to_not_be_null (Column: Order ID)
   ✓ expect_column_values_to_be_between (Column: Qty)
   ✓ expect_column_values_to_be_between (Column: Amount)
   ✓ expect_column_values_to_be_in_set (Column: Status)
   ✓ expect_column_values_to_be_in_set (Column: Fulfilment)
   ✓ expect_column_values_to_be_in_set (Column: currency)
   ✓ expect_column_values_to_be_in_set (Column: ship-country)
   ✓ expect_column_values_to_match_regex (Column: Date)

🔍 Running Pydantic Validation...

   PYDANTIC VALIDATION SUMMARY
   Overall : ✅ PASSED
   Total Rows   : 128975
   Valid Rows   : 128975
   Invalid Rows : 0
   Error Count  : 0

   FINAL RESULT
   GE Validation      : ✅
   Pydantic Validation : ✅
   Overall             : ✅ ALL PASSED
